# Spatio-temporal coherence

Nous souhaitons nous assurer que les données décrivent un même phénomène physique sur une grille spatiale et temporelle cohérente, et cela, en maintenant une continuité et une exploitabilité pour l’analyse et la modélisation.

## 1. Cohérence temporelle

### 1.1 Vérifier la structure du temps

In [18]:
import xarray as xr

dsNOAA = xr.open_dataset("data/raw/sstNOAA20102019.nc")
dsCOPERNICUS = xr.open_dataset("data/raw/sstCOPERNICUS20102019.nc")

print(dsNOAA.time)
print(dsNOAA.time.encoding)

print("\n***************************************\n")

print(dsCOPERNICUS.time)
print(dsCOPERNICUS.time.encoding)

<xarray.DataArray 'time' (time: 3652)> Size: 29kB
array(['2010-01-01T00:00:00.000000000', '2010-01-02T00:00:00.000000000',
       '2010-01-03T00:00:00.000000000', ..., '2019-12-29T00:00:00.000000000',
       '2019-12-30T00:00:00.000000000', '2019-12-31T00:00:00.000000000'],
      shape=(3652,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 29kB 2010-01-01 2010-01-02 ... 2019-12-31
Attributes:
    long_name:     Time
    delta_t:       0000-00-01 00:00:00
    avg_period:    0000-00-01 00:00:00
    axis:          T
    actual_range:  [76701. 77065.]
{'dtype': dtype('float64'), 'zlib': False, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': False, 'complevel': 0, 'fletcher32': False, 'contiguous': True, 'chunksizes': None, 'source': '/home/mkkuu/Documents/GitHub/m1Project_SciML/data/raw/sstNOAA20102019.nc', 'original_shape': (3652,), '_FillValue': np.float64(nan), 'units': 'days since 1800-01-01', 'calendar': 'proleptic_gregorian'}

******

En comparant les axes temporels des deux jeux de données, on constate un léger décalage de borne temporelle. Copernicus incluant un jour supplémentaire au-delà de l'année 2019. Pour garantir la cohérence temporelle stricte entre les sources, nous allons simplement restreindre le jeu de donnée COPERNICUS à la période commune 2010-2019.

In [19]:
print(dsNOAA.time.equals(dsCOPERNICUS.time)) # False, confirmant l'inégalité des bornes temporelles

dsCOPERNICUS = dsCOPERNICUS.sel(time=slice("2010-01-01", "2019-12-31"))

print(dsNOAA.time.equals(dsCOPERNICUS.time)) # Testons de nouveau l'égalité des bornes temporelles

print(dsCOPERNICUS.time)

False
False
<xarray.DataArray 'time' (time: 3653)> Size: 29kB
array(['2010-01-01T00:00:00.000000000', '2010-01-02T00:00:00.000000000',
       '2010-01-03T00:00:00.000000000', ..., '2019-12-29T00:00:00.000000000',
       '2019-12-30T00:00:00.000000000', '2019-12-31T00:00:00.000000000'],
      shape=(3653,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 29kB 2010-01-01 2010-01-02 ... 2019-12-31
Attributes:
    standard_name:  time
    long_name:      Time
    axis:           T


On constate toujours une inégalité entre le nombre de points temporels dans les deux jeux de données. Il semblerait qu'il y ait toujours un point temporel supplémentaire (nous voulons avoir 3652 points correspondant à 8 années à 365 jours et 2 années bissextiles à 366 jours, soit 3652 jours). Explorons l'hypothèse d'un éventuel doublon en vérifiant le pas de temps.

In [20]:
import pandas as pd

dtNOAA = pd.to_datetime(dsNOAA.time.values).to_series().diff().dropna().unique()
dtCOPERNICUS  = pd.to_datetime(dsCOPERNICUS.time.values).to_series().diff().dropna().unique()

print(dtNOAA, dtCOPERNICUS)


<TimedeltaArray>
['1 days']
Length: 1, dtype: timedelta64[ns] <TimedeltaArray>
['1 days', '0 days']
Length: 2, dtype: timedelta64[ns]
